# General Settings and Imports

In [136]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [137]:
#general stuff
import pandas as pd
import numpy as np
import json
from pprint import pprint
import datetime
import os

#bfs
from collections import deque

#ofac stuff
import csv
import requests
#import json
import xmltodict
from collections import defaultdict

#dark web stuff
#import pyarrow.parquet as pq
#from pybloom_live import ScalableBloomFilter
import dask.dataframe as dd

#for graphsense tag data extraction
#from fastparquet import ParquetFile
#import snappy

#for flipside.xyz (SQL Databse)
#from flipside import Flipside

#graphsense stuff
import graphsense
from graphsense.api import addresses_api, bulk_api, entities_api, general_api, tags_api
from graphsense.model.neighbor_addresses import NeighborAddresses

#coinbase stuff
#import json, requests
import hmac, hashlib, time
from requests.auth import AuthBase

In [138]:
f = open('config.json')
config = json.load(f)
f.close()

CURRENCY = 'eth'
ADDRESS = '0x49bd56B275395130B41D67E1304b2d49F8A88725'

configuration = graphsense.Configuration(
    host = config['graphsense']['host'],
    api_key = {'api_key': config['graphsense']['api_key']})

#with graphsense.ApiClient(configuration) as api_client:
    #api_instance = general_api.GeneralApi(api_client)
    #api_response = api_instance.get_statistics()
    #pprint(api_response)

# Associated Wallets Module

In [139]:
class AssociatedWallets():
    
    def simple_neighbors(self, address, direction):
        with graphsense.ApiClient(configuration) as api_client:
            # Create an instance of the API class
            api_instance = addresses_api.AddressesApi(api_client)
            currency = CURRENCY # str | The cryptocurrency code (e.g., eth)
            address = ADDRESS # str | The cryptocurrency address
            direction = "out" # str | Incoming or outgoing neighbors
            only_ids = [
                "only_ids_example",
            ] # [str] | Restrict result to given set of comma separated addresses (optional)
            include_labels = False # bool | Whether to include labels of first page of address tags (optional) if omitted the server will use the default value of False
            page = "page_example" # str | Resumption token for retrieving the next page (optional)
            pagesize = 10 # int | Number of items returned in a single page (optional)

            # example passing only required values which don't have defaults set
            try:
                # Get an address's neighbors in the address graph
                api_response = api_instance.list_address_neighbors(currency, address, direction)
                return api_response
            except graphsense.ApiException as e:
                print("Exception when calling AddressesApi->list_address_neighbors: %s\n" % e)

            # example passing only required values which don't have defaults set
            # and optional values
            #try:
                # Get an address's neighbors in the address graph
                #api_response = api_instance.list_address_neighbors(currency, address, direction, only_ids=only_ids, include_labels=include_labels, page=page, pagesize=pagesize)
                #pprint(api_response)
            #except graphsense.ApiException as e:
                #print("Exception when calling AddressesApi->list_address_neighbors: %s\n" % e)

    #Utility function to get neighbors of address        
    def get_addr_neighbors(self, address, direction):

        if (direction == 'out'):
            degree = 'out_degree'
            address_degree = 'address_out_degree'
        else:
            degree = 'in_degree'
            address_degree = 'address_in_degree'

        with graphsense.ApiClient(configuration) as api_client:
            try:
                api_instance = bulk_api.BulkApi(api_client)
                print(f"get_addr_neighbors of {address}")
                operation = "list_address_neighbors"
                body = {'address': [address], 'direction': direction}

                df_address_neighbors = pd.read_csv(api_instance.bulk_csv(CURRENCY, operation, body=body,
                                                                    num_pages=1, _preload_content=False))
                
                df_address_neighbors = df_address_neighbors \
                    .loc[(df_address_neighbors['_error'] != 'not found') &
                        (df_address_neighbors['_info'] != 'no data')].reset_index(drop=True)
                
                if df_address_neighbors.empty:
                    df_address_neighbors.columns = ['address', 'entity', degree]
                    return df_address_neighbors
                
                print(df_address_neighbors.columns)  # Print column names

                
                df_address_neighbors = df_address_neighbors \
                    .rename(columns={'address_address': 'address', 
                                    'address_entity': 'entity',
                                    address_degree: degree})

                return df_address_neighbors[['address', 'entity', degree]]
                
            except graphsense.ApiException as e:
                print("Exception when calling Bulk Api: %s\n" % e)

    # The Breadth-First Search algorithm:
    def bfs_neighbors(self, seed_address, max_depth, max_degree, verbose, direction):

        if (direction == 'out'):
            degree = 'out_degree'
            #address_degree = 'address_out_degree'
        else:
            degree = 'in_degree'
            #address_degree = 'address_in_degree'
        
        # collect neighbors
        neighbors = []
        for i in range (0, max_depth):
            neighbors.insert(i, [])
        neighbors[0].append(seed_address)

        #keep track of depth
        levels = {seed_address: 0}
                
        # record visited addresses and entities
        visited_addresses = set([seed_address])
        
        # maintain a queue of addresses
        queue = deque([(seed_address, 0)])

        while(queue):

            # get first address from the queue
            addr, level = queue.popleft()            

            # retrieve address neighbors
            df_neighbors = self.get_addr_neighbors(addr, direction)

            # continue with neighbors out_degree < max_outdegree
            for index, neighbor in df_neighbors.iterrows():

                 # stop if address has already been visited
                if(neighbor['address'] in visited_addresses):
                    if verbose:
                        print(addr, end=' ') 
                        print("STOP | same address")
                    continue
                                
                # stop if max depth is reached
                if level + 1 == max_depth:
                    print(level)
                    if verbose:
                        print(addr, end=' ') 
                        print("STOP | max depth")
                    continue

                neighbors[level+1].append(neighbor['address'])

                # stop if address out_degree exceeds threshold
                if(neighbor[degree] > max_degree):
                    if verbose:
                        print(addr, end=' ') 
                        print("STOP | max degree")
                    continue
                
                queue.append((neighbor['address'], level + 1))
                visited_addresses.add(neighbor['address'])
                levels[neighbor['address']] = level + 1
                    
            if len(queue) == 0:
                return neighbors

# Blacklist Module

In [140]:
class Blacklist():
    
    def scrape_ofac(self):

        coins = []
        myCoins = {}
        sdn = defaultdict(list)
        filename = 'sdn2.xml'
        URL = "https://www.treasury.gov/ofac/downloads/sanctions/1.0/sdn_advanced.xml"

        response = requests.get(URL)
        with open(filename, 'wb') as file:
            file.write(response.content)

        xml_data = open(filename, 'r').read()  # Read file
        d = xmltodict.parse(xml_data)

        for i in d['Sanctions']['ReferenceValueSets']['FeatureTypeValues']['FeatureType']:
            if 'Digital Currency Address' in i['#text']:
                coins.append(i['@ID'])
                myCoins[i['@ID']] =  i['#text'].replace('Digital Currency Address - ','')
            #   print(i['@ID'],'-',i['#text'])

        for i in d['Sanctions']['DistinctParties']['DistinctParty']:
            if 'Feature' in i['Profile'].keys():
                for j in i['Profile']['Feature']:
                    if '@FeatureTypeID' in j:
                        if type(j) is not str:
                            if str(j['@FeatureTypeID']) in coins:
                            #   print(j)
                            #   print(j['FeatureVersion']['VersionDetail']['#text'])
                                sdn[myCoins[j['@FeatureTypeID']]].append(j['FeatureVersion']['VersionDetail']['#text'])            
                            #    break
            
        with open('results/sdn.json', 'w') as fp:
            json_data = json.dump(sdn, fp)
        fp.close()

    def check_membership_ofac(self, currency, address, json_data):

        currency_set = set(json_data[currency])
        return address in currency_set
    
    def check_dark_web(self, address, parent_directory):
        
        # Iterate through each folder in the parent directory
        for folder in os.listdir(parent_directory):

            folder_path = os.path.join(parent_directory, folder)

            # Check if the item in the directory is a folder
            if os.path.isdir(folder_path):

                # Iterate through all Parquet files in the folder
                for file_name in os.listdir(folder_path):

                    if file_name.endswith(".parquet"):

                        file_path = os.path.join(folder_path, file_name)
                        # Read the Parquet file into a Dask DataFrame
                        ddf = dd.read_parquet(file_path, blocksize="400mb")
                        print(ddf.head())

                        # Check if the BTC address exists in the 'Address' column
                        check = ddf['address'].isin([address]).compute()

                        if check.any():
                            return True
                        
        return False


# Exchanges Module

In [141]:
class Exchange():
    
    def __init__(self, depth, direction):
        self.DEPTH = depth
        self.DIRECTION = direction

    def get_ex_tags(self, address, type):

        # Enter a context with an instance of the API client
        with graphsense.ApiClient(configuration) as api_client:
            # Create an instance of the API class
            api_instance = addresses_api.AddressesApi(api_client)

            currency = CURRENCY # str | The cryptocurrency code (e.g., btc)
            #address = ADDRESS # str | The cryptocurrency address
            page = "page_example" # str | Resumption token for retrieving the next page (optional)
            pagesize = 10 # int | Number of items returned in a single page (optional)
            include_tags = True

            # example passing only required values which don't have defaults set
            try:
                # Get attribution tags for a given address
                api_response = api_instance.list_tags_by_address(currency, address.lower())
                exchange_data = api_response
                
                exchange_tags = []

                for entry in exchange_data['address_tags']:
                    actor = entry.get('actor', '')
                    category = entry.get('category', '')
                    label = entry.get('label', '')
                    if ((actor or label) and (type == 'cex' and (category == 'exchange' or category == 'market'))) or ((actor or label) and type == 'dex' and category == 'defi_dex'):
                        exchange_tags.append({
                            'actor': actor,
                            'category': category,
                            'currency': entry.get('currency', ''),
                            'label': label
                        })

                return exchange_tags
                
            except graphsense.ApiException as e:
               print("Exception when calling AddressesApi->list_tags_by_address: %s\n" % e)
    
    def get_ex_used(self, type):

        #hier einfach neighbors von ADDRESS finden mit der self.depth dann jedes von denen in get_cex_tags und dann falls result nicht empty ist einf in einer liste alles speichern und dann printen
        
        ass_wallets = AssociatedWallets()
        max_degree = 7
        verbose = True
        neighbors = ass_wallets.bfs_neighbors(ADDRESS, self.DEPTH, max_degree, verbose, self.DIRECTION)

        exchanges_used = []

        for arr_level in neighbors:
            for address in arr_level:
                temp_exchange_tags = self.get_ex_tags(address, type)
                if temp_exchange_tags:
                    exchanges_used.append(temp_exchange_tags)

        return exchanges_used

# Proof of Funds Module

In [142]:
# Create custom authentication for Coinbase API
class CoinbaseWalletAuth(AuthBase):
    def __init__(self, api_key, secret_key):
        self.api_key = api_key
        self.secret_key = secret_key

    def __call__(self, request):
        timestamp = str(int(time.time()))
        message = timestamp + request.method + request.path_url + (request.body or '')
        signature = hmac.new(self.secret_key, message, hashlib.sha256).hexdigest()

        request.headers.update({
            'CB-ACCESS-SIGN': signature,
            'CB-ACCESS-TIMESTAMP': timestamp,
            'CB-ACCESS-KEY': self.api_key,
        })
        return request

class PoF():

    #Candles data Coinbase

    """Each bucket is an array of the following information:

    time: bucket start time
    low: lowest price during the bucket interval
    high: highest price during the bucket interval
    open: opening price (first trade) in the bucket interval
    close: closing price (last trade) in the bucket interval
    volume: volume of trading activity during the bucket interval"""
    
    def coinbase_data(self, type, start_date=datetime.date.today()):

        # Before implementation, set environmental variables with the names API_KEY and API_SECRET

        f = open('config.json')
        config = json.load(f)
        f.close()   

        #API_KEY = config['coinbase']['api_key']
        #API_SECRET = config['coinbase']['api_secret']
        #auth = CoinbaseWalletAuth(API_KEY, API_SECRET)

        if type == 'buy':
            api_url = config['coinbase']['buyprice_endpoint']
        elif type == 'sell':
            api_url = config['coinbase']['sellprice_endpoint']
        elif type == 'exchange':
            api_url = config['coinbase']['exchangeprice_endpoint']
        elif type == 'spot':
            api_url = config['coinbase']['spotprice_endpoint']
        elif type == 'candles':
            api_url = config['coinbase']['candles_endpoint']

        # Convert the start date string to a datetime object
        #start_date_datetime = datetime.datetime.strptime(start_date, "%Y-%m-%d")

        # Calculate the end date by adding one day to the start date
        #end_date_datetime = start_date_datetime + datetime.timedelta(days=1)

        # Convert the end date to a string in the same format
        #end_date = end_date_datetime.strftime("%Y-%m-%d")

        # Parameters for the API request
        params_spot = {
            "date": start_date
        }

        # Parameters for the API request
        params_candles = {
            "granularity": 86400,  # Daily interval
            "start": start_date,
            "end": start_date
        }

        # Get current user
        #r = requests.get(api_url + 'user', auth=auth)
        #print(r.json())
        # {u'data': {u'username': None, u'resource': u'user', u'name': u'User'...

        # Send funds
        """tx = {
            'type': 'send',
            'to': 'user@example.com',
            'amount': '10.0',
            'currency': 'USD',
        }"""

        #r = requests.post(api_url + 'accounts/primary/transactions', json=tx, auth=auth)
        #print(r.json())
        # {u'data': {u'status': u'pending', u'amount': {u'currency': u'BTC'...

        if (len(start_date) == 0):
            response = requests.get(api_url)
        else:
            if type == 'spot':
                response = requests.get(api_url, params=params_spot)
            elif type == 'candles':
                response = requests.get(api_url, params=params_candles)

        # Process the response data
        if response.status_code == 200:
            return response.json()
        else:
            print("Request failed:", response.status_code)

    
    #Klines data Binance
    """[
        [
            1499040000000,      // Kline open time
            "0.01634790",       // Open price
            "0.80000000",       // High price
            "0.01575800",       // Low price
            "0.01577100",       // Close price
            "148976.11427815",  // Volume
            1499644799999,      // Kline Close time
            "2434.19055334",    // Quote asset volume
            308,                // Number of trades
            "1756.87402397",    // Taker buy base asset volume
            "28.46694368",      // Taker buy quote asset volume
            "0"                 // Unused field, ignore.
        ]
        ]"""
    
    def binance_data(self, type='avgprice', start_date=datetime.date.today(), end_date=datetime.datetime.today(), symbol='ETHUSDT'):

        f = open('config.json')
        config = json.load(f)
        f.close()

        if type == 'avgprice':
            api_url = config['binance']['avgprice_endpoint']
        elif type == 'exchange':
            api_url = config['binance']['exchangeprice_endpoint']
        elif type == 'klines':
            api_url = config['binance']['klines_endpoint']
        elif type == '24hrstats':
            api_url = config['binance']['24hrstats_endpoint']
        
        # Convert start and end dates to Unix timestamps (in milliseconds)

        start_timestamp = int(datetime.datetime.strptime(start_date, "%Y-%m-%d").timestamp()) * 1000
        #end_timestamp = int(datetime.datetime.strptime(end_date, "%Y-%m-%d").timestamp()) * 1000

        # Parameters for the API request
        params = {
            "symbol": symbol,
            "interval": "1d",  # Daily interval
            "startTime": start_timestamp
            #"endTime": end_timestamp
        }

        response = requests.get(api_url, params=params)

        # Process the response data
        if response.status_code == 200:
            return response.json()
        else:
            print("Request failed:", response.status_code)
        
        
    def binance_table(self, ticker, interval='4h', limit=500, start='01-01-2023'):

        """
        interval: str tick interval - 4h/1h/1d ...
        """

        columns = ['open_time','open', 'high', 'low', 'close', 'volume','close_time', 'qav','num_trades','taker_base_vol','taker_quote_vol', 'ignore']
        start = int(datetime.datetime.timestamp(pd.to_datetime(start))*1000)
        api_url = f'https://www.binance.com/api/v3/klines?symbol={ticker}&interval={interval}&limit={limit}&startTime={start}'

        data = pd.DataFrame(requests.get(api_url).json(), columns=columns, dtype=float)

        data.index = [pd.to_datetime(x, unit='ms').strftime('%Y-%m-%d %H:%M:%S') for x in data.open_time]

        usecols=['open', 'high', 'low', 'close', 'volume', 'qav','num_trades','taker_base_vol','taker_quote_vol']
        
        data = data[usecols]

        return data
    
    
    def check_price(self, exchange, symbol, buy_date, sell_date, buy_amount, sell_amount, profit_or_loss):
        
        if exchange == 'coinbase':
            buy_price_data = self.coinbase_data('candles', buy_date)
            sell_price_data = self.coinbase_data('candles', sell_date)

            buy_price = float(buy_price_data[0][3])
            sell_price = float(sell_price_data[0][3])

            buy_total = buy_price * buy_amount
            sell_total = sell_price * sell_amount
            
        elif exchange == 'binance':
            
            buy_price_data = self.binance_data('klines', buy_date, symbol)
            sell_price_data = self.binance_data('klines', sell_date, symbol)

            buy_price = float(buy_price_data[0][1])
            sell_price = float(sell_price_data[0][1])

            buy_total = buy_price * buy_amount
            sell_total = sell_price * sell_amount

        difference = abs(abs(float(buy_price) - float(sell_price)) - abs(profit_or_loss))
        
        if abs(float(buy_total) - float(sell_total)) == abs(profit_or_loss):
            print("Actual profit/loss: " + str(float(sell_total) - float(buy_total)))
            print("Difference between reported profit/loss and actual profit/loss value: " + str(difference))
            return True
        else:
            print("Actual profit/loss: " + str(float(sell_total) - float(buy_total)))
            print("Difference between reported profit/loss and actual profit/loss value: " + str(difference))            
            return False



# NFTs Module

In [143]:
class NFT():

    def get_nft_transfers(self, address, page=1, offset=100):

        f = open('config.json')
        config = json.load(f)
        f.close()

        params = {
            "module": "account",
            "action": "tokennfttx",
            "address": address,
            "page": page,
            "offset": offset,
            "sort": "asc",
            "apikey": config['etherscan']['api_key']
         }
        
        response = requests.get(config['etherscan']['host'], params=params)
        data = response.json()

        if data['status'] != "1" or 'result' not in data:
            print(f"Error fetching transactions for address {address}: {data['message']}")
            return []

        return data['result']
    
    def get_held_nfts(self, address):
        
        transfers = self.get_nft_transfers(address)

        incoming_nfts = {tx['tokenID']: tx for tx in transfers if tx['to'] == address.lower()}
        outgoing_nfts = {tx['tokenID']: tx for tx in transfers if tx['from'] == address.lower()}

        held_nfts = {tid: tx for tid, tx in incoming_nfts.items() if tid not in outgoing_nfts}

        return list(held_nfts.values())


# Main Section

In [144]:
# Function calls

#ass_wallets = AssociatedWallets(CURRENCY)

#direction = 'out'

#addr_neigh = ass_wallets.bfs_neighbors(ADDRESS, 4, 10, True, direction)
#addr_simple_neigh = ass_wallets.simple_neighbors(ADDRESS, direction)

#with open("results/addresses_" + direction + ".txt", "w") as outfile:
    #outfile.write(str(addr_neigh))

#blacklist = Blacklist()
#blacklist.ofac()
#print(blacklist.check_membership('ETH', ADDRESS, json_data))
#blacklist.check_dark_web('1DcU28QbeUiJVcSS8sZCf4ixeYXG26zEWN', 'blacklist_data/address_cluster.parquet')

 # Open the JSON file
#with open('results/sdn.json', 'r') as file:
    # Load the JSON data
    #json_data = json.load(file)


#ex = Exchange(3, 'in')
#exchanges_used = ex.get_ex_used('dex')
#exchanges_used = ex.get_ex_tags('0x12459C951127e0c374FF9105DdA097662A027093', 'dex')
#pprint(exchanges_used)

#pof = PoF()

#print(pof.coinbase_data('candles', '2018-01-01'))
#pof.binance_data('klines', '2023-01-13', 'ETHUSDT')

#pprint(pof.binance_table('ETHUSDT', '1h'))
#pof.check_price('coinbase', 'ETHUSDT', '2020-01-01', '2023-09-04', 1, 1, 1507.1100000000001)

nft = NFT()
nfts_list = nft.get_held_nfts(ADDRESS)
for nft in nfts_list:
    print(nft)

{'blockNumber': '14710242', 'timeStamp': '1651658186', 'hash': '0x78c490ed3bba9b6fd9ec5e1636365e904a78869966274b67bb58aa2d66979d92', 'nonce': '0', 'blockHash': '0x12bfae624d6022ef6a88bfe51ec17984ff87f9385bde9100c50815eb9e9eb213', 'from': '0x981b749a0ddca1c8f2ddc44a52b57d89e2d51344', 'contractAddress': '0x25ed58c027921e14d86380ea2646e3a1b5c55a8b', 'to': '0x49bd56b275395130b41d67e1304b2d49f8a88725', 'tokenID': '3876', 'tokenName': 'Devs for Revolution', 'tokenSymbol': 'DEVS', 'tokenDecimal': '0', 'transactionIndex': '191', 'gas': '355560', 'gasPrice': '30228736586', 'gasUsed': '256124', 'cumulativeGasUsed': '23328673', 'input': 'deprecated', 'confirmations': '3369022'}
{'blockNumber': '15062025', 'timeStamp': '1656750548', 'hash': '0x818df6759851c4dac83ed2f47d0290b5f6e56b7d0d346ea0a8b766cda67b460c', 'nonce': '2', 'blockHash': '0x2e574e6b7a72768880c559027c3b4cdaea13503c28e509a3a33d91a9f88cc9d7', 'from': '0x283af0b28c62c092c9727f1ee09c02ca627eb7f5', 'contractAddress': '0x57f1887a8bf19b14fc